In [ ]:
import pandas as pd

In [ ]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
#dataframe
df = pd.DataFrame()

In [ ]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

In [ ]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

All Antidepressants Ingredients

In [ ]:
#loop array to get complete list of drugs
df_antidep = pd.DataFrame()
drugs = []
for drug in drugs:
    self_df = query_self(drug)
    merg_df = query_drug(drug)
    df_antidep = pd.concat([df_antidep, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_antidep = pd.concat([df_antidep, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_antidep

In [ ]:
#Export
df_antidep.to_excel("New_Complete_Antidepressants_List.xlsx", index=False)